# Community FBA models

In [1]:
using COBREXA

Here we will construct a community FBA model of two  *E. coli* "core" models
that can interact by exchanging selected metabolites. To do this, we will need
the model, which we can download if it is not already present.

In [2]:
import Downloads: download

!isfile("e_coli_core.json") &&
    download("http://bigg.ucsd.edu/static/models/e_coli_core.json", "e_coli_core.json")

false

Additionally to COBREXA and the model format package, we will need a solver
-- let's use GLPK here:

In [3]:
import JSONFBCModels
import GLPK
import ConstraintTrees as C

model = load_model("e_coli_core.json")

JSONFBCModels.JSONFBCModel(#= 95 reactions, 72 metabolites =#)

Community models work by joining its members together through their exchange
reactions, weighted by the abundance of each microbe. These exchange reactions
are then linked to an environmental exchange. For more theoretical details,
see "Gottstein, et al, 2016, Constraint-based stoichiometric modelling from
single organisms to microbial communities, Journal of the Royal Society
Interface".

## Building a community of two *E. coli*s

Here we will construct a simple community of two interacting microbes. To do
this, we need to import the models. We will represent the models only as
constraint trees, because it is easier to build the model explicitly than
rely on an opaque one-shot function.

In [4]:
ecoli1 = flux_balance_constraints(model, interface = :sbo)
ecoli2 = flux_balance_constraints(model, interface = :sbo)

ConstraintTrees.Tree{ConstraintTrees.Constraint} with 4 elements:
  :flux_stoichiometry => ConstraintTrees.Tree{ConstraintTrees.Constraint}(#= 72…
  :fluxes             => ConstraintTrees.Tree{ConstraintTrees.Constraint}(#= 95…
  :interface          => ConstraintTrees.Tree{ConstraintTrees.Constraint}(#= 2 …
  :objective          => ConstraintTrees.Constraint(ConstraintTrees.LinearValue…

Since the models are usually used in a mono-culture context, the glucose input
for each individual member is limited. We need to undo this limitation, and
rather rely on the constrained environmental exchange reaction (and the bounds
we set for it earlier).

In [5]:
ecoli1.fluxes.EX_glc__D_e.bound = C.Between(-1000.0, 1000.0)
ecoli2.fluxes.EX_glc__D_e.bound = C.Between(-1000.0, 1000.0)

ConstraintTrees.Between(-1000.0, 1000.0)

To make the community interesting, we can limit different reactions in both
members to see how the  models cope together:

In [6]:
ecoli1.fluxes.CYTBD.bound = C.Between(-10.0, 10.0)
ecoli2.fluxes.ACALD.bound = C.Between(-5.0, 5.0)

ConstraintTrees.Between(-5.0, 5.0)

Because we created the trees with interfaces, we can connect them easily to
form a new model with the interface. For simplicity, we use the
interface-scaling functionality of [`interface_constraints`](@ref
ConstraintTrees.interface_constraints) to bring in cFBA-like community member
abundances:

In [7]:
cc = interface_constraints(
    :bug1 => (ecoli1, ecoli1.interface, 0.2),
    :bug2 => (ecoli2, ecoli2.interface, 0.8),
)

ConstraintTrees.Tree{ConstraintTrees.Constraint} with 4 elements:
  :bug1              => ConstraintTrees.Tree{ConstraintTrees.Constraint}(#= 4 e…
  :bug2              => ConstraintTrees.Tree{ConstraintTrees.Constraint}(#= 4 e…
  :interface         => ConstraintTrees.Tree{ConstraintTrees.Constraint}(#= 2 e…
  :interface_balance => ConstraintTrees.Tree{ConstraintTrees.Constraint}(#= 2 e…

To make the community behave as expected, we need to force equal (scaled)
growth of all members:

In [8]:
cc *=
    :equal_growth^equal_value_constraint(
        cc.bug1.fluxes.BIOMASS_Ecoli_core_w_GAM,
        cc.bug2.fluxes.BIOMASS_Ecoli_core_w_GAM,
    )

ConstraintTrees.Tree{ConstraintTrees.Constraint} with 5 elements:
  :bug1              => ConstraintTrees.Tree{ConstraintTrees.Constraint}(#= 4 e…
  :bug2              => ConstraintTrees.Tree{ConstraintTrees.Constraint}(#= 4 e…
  :equal_growth      => ConstraintTrees.Constraint(ConstraintTrees.LinearValue(…
  :interface         => ConstraintTrees.Tree{ConstraintTrees.Constraint}(#= 2 e…
  :interface_balance => ConstraintTrees.Tree{ConstraintTrees.Constraint}(#= 2 e…

Now we can simulate the community growth by optimizing the new "interfaced"
biomass:

In [9]:
optimized_cc = optimized_values(
    cc,
    objective = cc.interface.biomass.BIOMASS_Ecoli_core_w_GAM.value,
    optimizer = GLPK.Optimizer,
)

ConstraintTrees.Tree{Float64} with 5 elements:
  :bug1              => ConstraintTrees.Tree{Float64}(#= 4 elements =#)
  :bug2              => ConstraintTrees.Tree{Float64}(#= 4 elements =#)
  :equal_growth      => 0.0
  :interface         => ConstraintTrees.Tree{Float64}(#= 2 elements =#)
  :interface_balance => ConstraintTrees.Tree{Float64}(#= 2 elements =#)

We can now e.g. observe the differences in individual pairs of exchanges:

In [10]:
C.zip(
    tuple,
    optimized_cc.bug1.interface.exchanges,
    optimized_cc.bug2.interface.exchanges,
    Tuple{Float64,Float64},
)

ConstraintTrees.Tree{Tuple{Float64, Float64}} with 20 elements:
  :EX_ac_e     => (119.498, 340.912)
  :EX_acald_e  => (0.0, 0.0)
  :EX_akg_e    => (0.0, 0.0)
  :EX_co2_e    => (423.72, 49.1276)
  :EX_etoh_e   => (681.106, 0.0)
  :EX_for_e    => (561.538, 340.123)
  :EX_fru_e    => (0.0, 0.0)
  :EX_fum_e    => (0.0, 2.13163e-14)
  :EX_glc__D_e => (-543.854, -291.288)
  :EX_gln__L_e => (0.0, 0.0)
  :EX_glu__L_e => (-3.55271e-15, 0.0)
  :EX_h2o_e    => (-141.56, 164.953)
  :EX_h_e      => (1000.0, 1000.0)
  :EX_lac__D_e => (0.0, 0.0)
  :EX_mal__L_e => (0.0, 0.0)
  :EX_nh4_e    => (-86.7025, -86.7025)
  :EX_o2_e     => (-5.0, -200.806)
  :EX_pi_e     => (-58.4933, -58.4933)
  :EX_pyr_e    => (0.0, 0.0)
  :EX_succ_e   => (0.0, 0.0)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*